# Running Prompt Functions Inline


Initial configuration for the notebook to run properly.

In [2]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

The [previous notebook](./02-running-prompts-from-file.ipynb)
showed how to define a semantic function using a prompt template stored on a file.

In this notebook, we'll show how to use the Semantic Kernel to define functions inline with your python code. This can be useful in a few scenarios:

- Dynamically generating the prompt using complex rules at runtime
- Writing prompts by editing Python code instead of TXT files.
- Easily creating demos, like this document

Prompt templates are defined using the SK template language, which allows to reference variables and functions. Read [this doc](https://aka.ms/sk/howto/configurefunction) to learn more about the design decisions for prompt templating.

For now we'll use only the `{{$input}}` variable, and see more complex templates later.

Almost all semantic function prompts have a reference to `{{$input}}`, which is the default way
a user can import content from the context variables.


Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):


Let's define our kernel for this example.

In [3]:
from semantic_kernel.kernel import Kernel

kernel = Kernel()

In [4]:
from services import Service

from service_settings import ServiceSettings

service_settings = ServiceSettings.create()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


We now configure our Chat Completion service on the kernel.

In [5]:
# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
            env_file_path="../.env",
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
            env_file_path="../.env",
        ),
    )

Let's use a prompt to create a semantic function used to summarize content, allowing for some creativity and a sufficient number of tokens.

The function will take in input the text to summarize.


In [6]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig

prompt = """{{$input}}
Summarize the content above.
"""

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-4o",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="summarize",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings,
)

summarize = kernel.add_function(
    function_name="summarizeFunc",
    plugin_name="summarizePlugin",
    prompt_template_config=prompt_template_config,
)

Set up some content to summarize, here's an extract about Demo, an ancient Greek poet, taken from Wikipedia (https://en.wikipedia.org/wiki/Demo_(ancient_Greek_poet)).


In [18]:
input_text = """
데모(고대 그리스 시인)
무료 백과 사전, 위키 백과에서
데모 또는 다모(그리스어: Δεμώ, Δαμώ, 기원전 200년경)는 로마 시대의 그리스 여성으로, 그녀의 이름이 새겨진 멤논의 거상에 새겨진 한 편의 시로 유명합니다. 거기에는 자신을 뮤즈에게 바친 서정 시인으로 묘사하고 있지만, 그녀의 생애에 대해서는 알려진 바가 없습니다.[1]
정체성
데메테르의 전통적인 별칭인 그녀의 이름에서 알 수 있듯이 데모는 그리스인임이 분명합니다. 이 이름은 이집트 등 헬레니즘 세계에서 비교적 흔한 이름이었기 때문에 더 이상 신원을 확인할 수 없습니다. 그녀가 멤논의 거상을 방문한 날짜는 확실하게 알 수 없지만, 왼쪽 다리에 새겨진 내부 증거에 따르면 서기 196년 이후 어느 시점에 그곳에 시가 새겨진 것으로 보입니다.[2]
에피그램
멤논의 거상에는 여러 개의 낙서 비문이 새겨져 있습니다. 줄리아 발빌라가 쓴 세 개의 에피그램에 이어 네 번째 에피그램은 “데모” 또는 “다모”(그리스어 비문은 읽기 어려움)가 쓴 것으로 추정되는 우아한 커플트로, 뮤즈에게 헌정되었습니다.[2] 이 시는 전통적으로 발빌라의 작품으로 출판되었지만 내부 증거는 다른 저자를 시사합니다.[1]
이 시에서 데모는 멤논이 자신에게 특별한 존경심을 보였다고 설명합니다. 이에 대한 보답으로 데모는 영웅에게 시를 선물합니다. 이 에피그램의 마지막에서 그녀는 멤논을 언급하며 그의 힘과 거룩함을 회상하며 그의 신성한 지위를 강조합니다.[2]
데모는 줄리아 발빌라와 마찬가지로 인공적이고 시적인 에올리아 방언으로 글을 씁니다. 이 언어는 그녀가 호메로스의 시를 잘 알고 있었음을 나타냅니다. 예를 들어 '즐거운 선물을 안고'라는 문구는 일리아드와 오디세이 전반에 걸쳐 이 문구가 사용되었음을 암시합니다.[a][2][3]
"""

...and run the summary function:


In [19]:
summary = await kernel.invoke(summarize, input=input_text)

print(summary)

데모 또는 다모는 기원전 200년경의 로마 시대 그리스 여성 시인으로, 멤논의 거상에 새겨진 시로 유명합니다. 그녀는 뮤즈에게 자신을 바친 서정 시인으로 묘사되지만, 생애에 대해서는 거의 알려진 바가 없습니다. 데모는 그리스인으로 추정되며, 멤논의 거상을 방문하여 시를 새긴 시점은 서기 196년 이후로 보입니다. 멤논의 거상에는 여러 낙서 비문이 있는데, 데모의 시는 뮤즈에게 헌정된 우아한 커플트로 구성되어 있습니다. 이 시는 멤논의 신성한 지위를 강조하며, 데모는 호메로스의 시를 잘 알고 있었음을 나타내는 에올리아 방언을 사용했습니다.


# Using ChatCompletion for Semantic Plugins


You can also use chat completion models (like `gpt-35-turbo` and `gpt4`) for creating plugins. Normally you would have to tweak the API to accommodate for a system and user role, but SK abstracts that away for you by using `kernel.add_service` and `AzureChatCompletion` or `OpenAIChatCompletion`


Here's one more example of how to write an inline Semantic Function that gives a TLDR for a piece of text using a ChatCompletion model


In [20]:
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
            env_file_path="../.env",
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
            env_file_path="../.env",
        ),
    )

In [22]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings

prompt = """
{{$input}}

30단어 이내로 TLDR을 알려주세요.
"""

text = """
    1) 로봇은 사람에게 상해를 입히거나 부주의로 인해
    인간을 위험에 빠뜨릴 수 없습니다.

    2) 로봇은 다음과 같은 경우를 제외하고는 인간이 내린 명령에 복종해야 합니다.
    제 1 법칙과 상충되는 경우를 제외하고는 로봇은 인간의 명령을 따라야 합니다.

    3) 로봇은 자신의 존재를 보호해야 하며, 그러한 보호가 제1법칙 또는 제2법칙과 충돌하지 않는 한
    제1법칙 또는 제2법칙과 충돌하지 않는 한 로봇은 자신의 존재를 보호해야 합니다.
"""

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        # ai_model_id="gpt-35-turbo",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="tldr",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings,
)

tldr_function = kernel.add_function(
    function_name="tldrFunction",
    plugin_name="tldrPlugin",
    prompt_template_config=prompt_template_config,
)

summary = await kernel.invoke(tldr_function, input=text)

print(f"Output: {summary}")

Output: 로봇은 인간에게 해를 끼치지 않고, 인간의 명령을 따르되, 자신의 존재를 보호해야 합니다.
